In [1]:
from time import time
import os

import pandas as pd
import tensorflow as tf

# from tensorflow.keras.models import load_model # type: ignore
from keras.api.models import load_model
import docplex.mp.model as mp

from utils.explanations import *
from utils.milp import * 

In [2]:
def store_data(file_results: str, dict_results: list[dict]) -> None:
    """Auxiliary function to store the results of the MILP model in a CSV file.
    Args:
        file_results (str): The path to the CSV file.
        dict_results (list[dict]): The list of dictionaries with the results.
    """

    file_exists = os.path.exists(file_results)
    df_new = pd.DataFrame(dict_results)
    df_combined = (
        df_new
        if not file_exists
        else pd.concat([pd.read_csv(file_results), df_new], ignore_index=True)
    )
    df_combined.to_csv(file_results, index=False)



def linear_constraints_to_dict(linear_constraints: list[mp.LinearConstraint]) -> dict:
    """Auxiliary function to get a dictionary from a list of linear constraints.
    Args:
        exp (list[mp.LinearConstraint]): The list of linear constraints.
    Returns:
        dict: A dictionary with the left expression as the key and the right expression as the value.
    """

    return {
        str(e.left_expr): float(e.right_expr.constant)
        for _, e in enumerate(linear_constraints)
    }


def generate_layers_to_relax(
    neural_network: tf.keras.Model, layers_idx: list[int]
) -> list[bool]:
    """Auxiliary function to generate the layers to relax in the MILP model.
    Args:
        neural_network (tf.keras.Model): The neural network model.
        layers_idx (list[int]): The indexes of the layers to relax.
    Returns:
        list[bool]: A list of boolean values indicating which layers to relax.
    """
    layers_to_relax = [False] * len(neural_network.layers)
    for _, idx in enumerate(layers_idx):
        layers_to_relax[idx] = True
    return layers_to_relax

In [3]:



def benchmark_mod(results_path: str, layers_idx: list[int]) -> None:
    # definir array de datasets
    datasets = [
        {"nome": "iris", "n_classes": 3},
        {"nome": "wine", "n_classes": 3},
        {"nome": "breast_cancer", "n_classes": 2},
        {"nome": "glass", "n_classes": 5},
        {"nome": "digits", "n_classes": 10},
        # {"nome":"mnist", "n_classes":10}, # rodar individualmente
    ]

    # definir array de modelos
    modelos = [
        # "model_1layers_20neurons.h5",
        "model_2layers_20neurons.h5",
        "model_3layers_20neurons.h5",
    ]

    # definir array de deltas a seres utilizados
    deltas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

    # para cada dataset
    for dataset in datasets:
        data = pd.read_csv(f"datasets/{dataset['nome']}/data.csv")
        # para cada modelo do dataset
        for modelo in modelos:
            print(f"Dataset: {dataset['nome']}, Modelo: {modelo}", end="\r")

            # carregar modelo em formato h5
            model_h5 = load_model(
                f"datasets/{dataset['nome']}/models/{modelo}", compile=False
            )

            # 1 - gerar modelos MILP
            method = "fischetti"
            model_milp, output_bounds = codify_network(model_h5, data, method, True)

            model_milp_relaxed = relax_model(
                model=model_milp,
                neural_network=model_h5,
                layers_to_relax=generate_layers_to_relax(model_h5, layers_idx),
            )
            # para cada instancia do dataset
            for index, instance in data.iterrows():
                if index >= 100:  # type: ignore
                    break

                # Predicao
                network_input = tf.reshape(tf.constant(data.iloc[index, :-1]), (1, -1))  # type: ignore
                network_output = tf.argmax(
                    model_h5.predict(tf.constant(network_input), verbose=0)[0]  # type: ignore
                )
                # print(f"Dataset: {dataset['nome']}, Modelo: {modelo}, Instance: {index}, Output: {network_output}")
                # print(f"Dataset: {dataset['nome']}, Modelo: {modelo}, Instance: {index}, Output: {network_output}", end="\r")

                # Explicacao
                # init = time()
                # (explanation, _) = get_minimal_explanation(
                #     model_milp.clone(),
                #     network_input,
                #     network_output,
                #     n_classes=dataset["n_classes"],
                #     method=method,
                #     output_bounds=output_bounds,
                # )
                # end = time()

                # original = {
                #     "time_milp": end - init,
                #     "len_milp": len(explanation),
                #     "explanation": linear_constraints_to_dict(explanation),
                # }

                # Relaxed
                # init = time()
                # (explanation, _) = get_minimal_explanation(
                #     model_milp_relaxed.clone(),
                #     network_input,
                #     network_output,
                #     n_classes=dataset["n_classes"],
                #     method=method,
                #     output_bounds=output_bounds,
                # )
                # end = time()

                # relaxed = {
                #     "time_milp_relaxed": end - init,
                #     "len_milp_relaxed": len(explanation),
                #     "explanation": linear_constraints_to_dict(explanation),
                # }

                # relaxed = {"times": [], "lens": []}
                relaxed = {"times": [], "len_eq": [], "len_range": []}
                for delta in deltas:
                    init = time()

                    # (explanation, _) = get_explanation_relaxed(
                    #     mdl=model_milp_relaxed.clone(),
                    #     network_input=network_input,
                    #     network_output=network_output,
                    #     n_classes=dataset["n_classes"],
                    #     delta=delta,
                    # )
                    # delta = 0.5
                    (explanation, _) = get_explanation_range(
                        mdl=model_milp_relaxed.clone(),
                        # mdl=model_milp.clone(),
                        network_input=network_input,
                        n_classes=dataset["n_classes"],
                        delta=delta,
                        network_output=network_output,
                    )
                    end = time()
                    relaxed["times"].append(end - init)
                    # relaxed["lens"].append(len(explanation))
                    relaxed["len_eq"].append(len(explanation["eq"])) #type: ignore
                    relaxed["len_range"].append(len(explanation["range"])) #type: ignore
                    
                store_data(
                    f"{results_path}/explanations_{dataset['nome']}.csv",
                    [
                        {
                            "dataset": dataset["nome"],
                            "modelo": modelo,
                            "instance": index,
                            # **{f"original_{k}": v for k, v in original.items()},
                            **{f"relaxed_{k}": v for k, v in relaxed.items()},
                        }
                    ],
                )

In [4]:
l = 0
path = os.path.join("results", "get_explanation_range", f"Layer {l} relaxed")
os.makedirs(path, exist_ok=True)
layers_idx = [l]
explanation = benchmark_mod(results_path=path, layers_idx=layers_idx)

In [5]:
explanation

TypeError: 'NoneType' object is not subscriptable